## Data Importing Notebook

In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression



from pathlib import Path

from api_keys import Openweather_api_key
from api_keys import Geoapify_key


#Here are the relative paths for assorted resource folders. Feel free to save to a file within these paths. 
APIDataPath = '../Resources/API_Data/'
AnalyzedDataPath = '../Resources/Data_Analysis/'
FiguresPath = '../Resources/Figures/'
#e.g. TVdata = f'{APIDataPath}TV_dat84_23.csv'


## Kevin Starts Here

In [49]:
#Kevin's code starts here

#TVdataPath = Path("../Resources/API_Data/TV_dat84_23.csv")
# reads in csv from Resources/API_Data folder
TVdataPath = APIDataPath + "TV_dat84_23.csv"
TVdataOriginal_df = pd.read_csv(TVdataPath)
TVdataOriginal_df.head(3)

,Series ID,Year,Period,Label,Value,3-Month Net Change,3-Month % Change
0,EIUIR41200,1984,M12,1984 Dec,132.2,N/A,N/A
1,EIUIR41200,1985,M03,1985 Mar,133.5,NaN,1.0
2,EIUIR41200,1985,M06,1985 Jun,129.5,NaN,-3.0


In [50]:
# removes unneeded columns
TV_data_Clean = TVdataOriginal_df.copy()
TV_data_Clean = TV_data_Clean.drop(columns=['Series ID', '3-Month Net Change'])
TV_data_Clean.head(2)

,Year,Period,Label,Value,3-Month % Change
0,1984,M12,1984 Dec,132.2,N/A
1,1985,M03,1985 Mar,133.5,1.0


In [51]:
# can eliminate first row if needed to resolve NA data prob
# takes Label column and coverts to Pandas date/time and reinserts back into DataFrame
#TV_data_First = TV_data_First.drop([0])
TV_data_date = pd.to_datetime(TV_data_Clean['Label'])
TV_data_Clean['Label'] = TV_data_date
TV_data_Clean = TV_data_Clean.rename(columns={"Label": "Date", "Period": "Month"})
TV_data_Clean.head(2)

,Year,Month,Date,Value,3-Month % Change
0,1984,M12,1984-12-01,132.2,N/A
1,1985,M03,1985-03-01,133.5,1.0


In [52]:
# elimates rows prior to set year and rests index to start bat 0
year = 2013
TV_data_2013 = TV_data_Clean[(TV_data_Clean['Date'].dt.year >= year)]
TV_data_2013.reset_index(drop = True, inplace = True)
TV_data_2013.head(3)

,Year,Month,Date,Value,3-Month % Change
0,2013,M01,2013-01-01,52.0,-3.0
1,2013,M02,2013-02-01,51.9,-0.6
2,2013,M03,2013-03-01,51.1,-2.1


In [53]:
TV_data_path = APIDataPath + 'TV_2013_cleaned.csv'
TV_data_path
TV_data_2013.to_csv(TV_data_path, index=False, header=True)

In [7]:
# 'Eggs' price table

In [54]:
# reads in csv from Resources/API_Data folder

EggsDataPath = APIDataPath + "eggs_84_23.csv"
EggsDataOriginal_df = pd.read_csv(EggsDataPath)
EggsDataOriginal_df.head(3)

,Series ID,Year,Period,Label,Value,3-Month % Change
0,APU0000708111,1984,M01,1984 Jan,1.301,35.2
1,APU0000708111,1984,M02,1984 Feb,1.324,32.9
2,APU0000708111,1984,M03,1984 Mar,1.153,1.9


In [55]:
# removes unneeded columns
EggsDataClean = EggsDataOriginal_df.copy() #original copy kept if needed
EggsDataClean = EggsDataClean.drop(columns=['Series ID'])
EggsDataClean.head(2)

,Year,Period,Label,Value,3-Month % Change
0,1984,M01,1984 Jan,1.301,35.2
1,1984,M02,1984 Feb,1.324,32.9


In [56]:
# takes Label column and coverts to Pandas date/time and reinserts back into DataFrame

Eggs_data_date = pd.to_datetime(EggsDataClean['Label'])
EggsDataClean['Label'] = Eggs_data_date
EggsDataClean = EggsDataClean.rename(columns={"Label": "Date", "Period": "Month"})
EggsDataClean.head(2)

,Year,Month,Date,Value,3-Month % Change
0,1984,M01,1984-01-01,1.301,35.2
1,1984,M02,1984-02-01,1.324,32.9


In [57]:
# elimates rows prior to set year and resets index to 0
year = 2013
EggsData_2013 = EggsDataClean[(EggsDataClean['Date'].dt.year >= year)]
EggsData_2013.reset_index(drop = True, inplace = True)
EggsData_2013.head(3)

,Year,Month,Date,Value,3-Month % Change
0,2013,M01,2013-01-01,1.933,-1.4
1,2013,M02,2013-02-01,1.965,0.1
2,2013,M03,2013-03-01,1.925,-4.1


In [109]:
#write cleaned data to csv file
Eggs_data_path_out = APIDataPath + 'Eggs_2013_cleaned.csv'
EggsData_2013.to_csv(Eggs_data_path_out, index=False, header=True)

In [ ]:
# New Automobiles and Automobile Parts Price Table

In [64]:
# reads in csv from Resources/API_Data folder

AutoDataPath = APIDataPath + "automobile_2016_2023.csv"
AutoDataOriginal_df = pd.read_csv(AutoDataPath)
AutoDataOriginal_df.head(3)

,Series ID,Year,Period,Label,Value,3-Month % Change
0,WPS5861,2016,M01,2016 Jan,99.0,N/A
1,WPS5861,2016,M02,2016 Feb,98.7,NaN
2,WPS5861,2016,M03,2016 Mar,96.8,NaN


In [65]:
# removes unneeded columns
AutoDataClean = AutoDataOriginal_df.copy() #original copy kept if needed
AutoDataClean = AutoDataClean.drop(columns=['Series ID'])
AutoDataClean.head(2)

,Year,Period,Label,Value,3-Month % Change
0,2016,M01,2016 Jan,99.0,N/A
1,2016,M02,2016 Feb,98.7,NaN


In [66]:
# takes Label column and coverts to Pandas date/time and reinserts back into DataFrame
# renames columns as needed
Auto_data_date = pd.to_datetime(AutoDataClean['Label'])
AutoDataClean['Label'] = Auto_data_date
AutoDataClean = AutoDataClean.rename(columns={"Label": "Date", "Period": "Month"})
AutoDataClean.head(2)

,Year,Month,Date,Value,3-Month % Change
0,2016,M01,2016-01-01,99.0,N/A
1,2016,M02,2016-02-01,98.7,NaN


In [67]:
#write cleaned data to csv file
Auto_data_path_out = APIDataPath + 'Automobile_2016_cleaned.csv'
AutoDataClean.to_csv(Auto_data_path_out, index=False, header=True)

In [ ]:
# Bananas Price Table

In [68]:
# reads in csv from Resources/API_Data folder
BananasDataPath = APIDataPath + "bananas_84_23.csv"
BananasDataOriginal_df = pd.read_csv(BananasDataPath)
BananasDataOriginal_df.head(3)

,Series ID,Year,Period,Label,Value,3-Month % Change
0,APU0000711211,1984,M01,1984 Jan,0.344,-10.6
1,APU0000711211,1984,M02,1984 Feb,0.373,14.1
2,APU0000711211,1984,M03,1984 Mar,0.373,18.4


In [70]:
#makes copy of original to work with and drops unneeded column(s)
BananasDataClean = BananasDataOriginal_df.copy()
BananasDataClean = BananasDataClean.drop(columns=['Series ID'])
BananasDataClean.head(2)

,Year,Period,Label,Value,3-Month % Change
0,1984,M01,1984 Jan,0.344,-10.6
1,1984,M02,1984 Feb,0.373,14.1


In [71]:
# takes Label column and coverts to Pandas date/time and reinserts back into DataFrame

Bananas_data_date = pd.to_datetime(BananasDataClean['Label'])
BananasDataClean['Label'] = Bananas_data_date
BananasDataClean = BananasDataClean.rename(columns={"Label": "Date", "Period": "Month"})
BananasDataClean.head(2)

,Year,Month,Date,Value,3-Month % Change
0,1984,M01,1984-01-01,0.344,-10.6
1,1984,M02,1984-02-01,0.373,14.1


In [98]:
# elimates rows prior to set year and resets index to 0
year = 2013
BananasData_2013 = BananasDataClean[(BananasDataClean['Date'].dt.year >= year)]
BananasData_2013.reset_index(drop = True, inplace = True)
BananasData_2013.head(3)

,Year,Month,Date,Value,3-Month % Change
0,2013,M01,2013-01-01,0.609,1.2
1,2013,M02,2013-02-01,0.611,1.8
2,2013,M03,2013-03-01,0.609,0.5


In [106]:
# code if needed to divide into quarter periods
#BananasData_2013a = BananasData_2013[(BananasData_2013['Month'] == "M09") | (BananasData_2013['Month'] == "M03") |
#                                   (BananasData_2013['Month'] == "M06") | (BananasData_2013['Month'] == "M12")]
#BananasData_2013a.head()

In [108]:
#write cleaned data to csv file
Bananas_data_path_out = APIDataPath + 'Bananas_2013_cleaned.csv'
BananasData_2013.to_csv(Bananas_data_path_out, index=False, header=True)

## Nelson's Starts here

In [9]:
# define a function to call the API and collect the data
def get_gas_prices():
    api_url = 'https://api.eia.gov/v2/series/?api_key=CaYN6JIcvNOO0ASoHuG1JYT1Hugfu98N3RFotIgL&series_id=PET.EMD_EPD2D_PTE_NUS_DPG.W'
    response = requests.get(api_url)
    data = json.loads(response.text)['series'][0]['data']
    gas_prices = pd.DataFrame(data, columns=['Year', 'Price'])
    gas_prices['Year'] = pd.to_datetime(gas_prices['Year'], format='%Y')
    gas_prices.set_index('Year', inplace=True)
    gas_prices.sort_index(inplace=True)
    return gas_prices

In [10]:
# create a scatter plot of the gas prices over time
plt.figure(figsize=(12, 6))
plt.scatter(gas_prices.index, gas_prices['Price'], alpha=0.5)
plt.xlabel('Year')
plt.ylabel('Gas Price ($)')
plt.title('Gas Prices in the USA over Time')
plt.show()

NameError: name 'gas_prices' is not defined

<Figure size 1200x600 with 0 Axes>

In [ ]:
# call the function to collect the data and create a data frame
gas_prices = get_gas_prices()
gas_prices.head()

In [ ]:
# create a linear regression to see how the gas prices have changed over time
X = gas_prices.index.map(lambda year: year.year).values.reshape(-1, 1)
y = gas_prices['Price'].values.reshape(-1, 1)
lr = LinearRegression().fit(X, y)
y_pred = lr.predict(X)

plt.figure(figsize=(12, 6))
plt.scatter(X, y, alpha=0.5)
plt.plot(X, y_pred, color='red')
plt.xlabel('Year')
plt.ylabel('Gas Price ($)')
plt.title('Gas Prices in the USA over Time')
plt.show()


In [ ]:
# create a bar plot to see the average gas prices by year
yearly_gas_prices = gas_prices.resample('Y').mean()

plt.figure(figsize=(12, 6))
plt.bar(yearly_gas_prices.index.year, yearly_gas_prices['Price'])
plt.xlabel('Year')
plt.ylabel('Average Gas Price ($)')
plt.title('Average Gas Prices in the USA by Year')
plt.show()


In [ ]:
# create a distribution map to see how the gas prices have varied across the USA
us_states = pd.read_csv('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json')
us_states['id'] = us_states['id'].apply(lambda x: x.lower())

state_gas_prices = gas_prices.groupby(gas_prices.index.year).mean()
state_gas_prices['State'] = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN']